In [1]:
import sqlite3
import json
import os

In [18]:
conn = sqlite3.connect("slr.db")

In [19]:
cursor = conn.cursor()

In [5]:
CREATE_SQL = """
   CREATE TABLE IF NOT EXISTS sign_videos (
        name TEXT PRIMARY KEY,
        video_array BLOB,
        verified INTEGER DEFAULT 0,
        true_label TEXT DEFAULT NULL
    ) 
"""

In [4]:
del_sql = """DROP TABLE sign_videos"""
cursor.execute(del_sql)
conn.commit()

In [6]:
cursor.execute(CREATE_SQL)
conn.commit()

In [7]:
with open(os.path.join('config', 'our_videos_indexes.json'), 'r', encoding='utf-8') as names_json:
    names_videos = json.load(names_json)

In [73]:
new_names_dir = {}

for sign, videos in names_videos.items():

    if not new_names_dir.get(sign):
        new_names_dir[sign] = []

    for i, video in enumerate(videos, 1):

        old_name_path = os.path.join(RESULTS_DIR, f"{video}.mp4")
        new_name_path = os.path.join(RESULTS_DIR, f"{sign.replace(' ', '_')}_video_{i}.mp4")
        # print(video)
    
        if os.path.exists(old_name_path):

            os.rename(old_name_path, new_name_path)
            new_names_dir[sign].append(new_name_path)

In [75]:
new_again = {}

for k, vals in new_names_dir.items():
    new_again[k] = [v.removeprefix('recordings\\') for v in vals]


In [9]:
%reload_ext autoreload
%autoreload 2

In [10]:
from utils.functions_ml import refactor_video_into_input

In [11]:
RESULTS_DIR = 'recordings'

In [12]:
for class_name, video_names in names_videos.items():
    for video_name in video_names:

        if os.path.exists(os.path.join(RESULTS_DIR, video_name)):
            refactored_numpy = refactor_video_into_input(os.path.join(RESULTS_DIR, video_name))

            try:
                bytes_ref = refactored_numpy.tobytes()
            except Exception as e:
                print(e)
            else:

                insert_sql = f"""
                    INSERT INTO sign_videos (name, video_array, verified, true_label) VALUES (?, ?, ?, ?)
                """
                
                try:
                    cursor.execute(insert_sql, (video_name, bytes_ref, 1, class_name))
                except Exception as e:
                    print(e)
                else:
                    conn.commit()

In [15]:
type(conn)

sqlite3.Connection

In [111]:
import numpy as np

In [20]:
cursor.execute('SELECT * FROM sign_videos')
res = cursor.fetchall()

In [22]:
res[-1]

('2023_10_24_21_05_05__SLR.webm',
 b'\xce\x10\x15?U\x07/?\xa1f)\xbfL\xf5\x7f?\xc88\x1d?\x19\x1f$?\xe9\x82(\xbf5\xf3\x7f?\xc1\xd1!?G\x11$?\x85\x93(\xbf+\xf2\x7f?}\x1a&?\xd2!$?\xee\xa5(\xbf\x9f\xf4\x7f?s2\x14?wA$?\xb4%\x15\xbfO\xf0\x7f?\xbb(\x12?\x99\x87$?0\xf1\x14\xbfV\xec\x7f?\xfd\xa6\x10?\n\xd4$?S\x0b\x15\xbf/\xed\x7f?\x19\xf22?xf+?h\xeb\xf8\xbet\xf6\x7f?w\xe3\x16?i\x07,?"2\r\xbe\x91\xe4\x7f?6\xb2\x1e?\x89\x01=?\x11/\x18\xbf\xdb\xf0\x7f?\xe4\x13\x14?\xa9\xa6<?_\xf2\xfc\xbe-\xe6\x7f?O\xc7[?\x15si?\x9c^\x0c\xbf\xe1\xee~?\xe6V\x0b?gjo?\xa8\x19.>\xeb\xf8}?\x7f\x0bj?\x8d\t\x9c?\xb47I\xbf\x89\xdc\x94>\xf1\x86\x05?\xda\x8c\x9e?\xed\xb4\'>\x95x\xdf<0\xc7X?\xd3\x90\xb9?U)\x83\xbf\xc4mU=\xd0\x96\xf5>\xbcM\xb6?K\x19\xe9\xbe\x8e\xdb\x85<\xa2\x97[?\x0f\xd7\xc3?\x1c\xd9\x93\xbf\xbe\x84\xbc=e\xfe\xe7>Z\x89\xbf?\xab\x8e\x0e\xbfe\xc4%=\xa6TP?\x0b;\xc3?<P\x96\xbf\x98\xd3\x1b>G\xdb\xf1>\x14\xfc\xbd?pO&\xbf\xd5\xa9\x92=\x02IN?\xe6W\xbf?M\xd6\x85\xbfJ\xa6\x05>f\xb9\xfa><\x87\xba?\x9b\xcf\x05\xbf"\xd5\x88=

In [114]:
numpy_array = np.frombuffer(res[-1][1], dtype=np.float32)

# Print the resulting NumPy array
print(numpy_array.reshape(25, -1))

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.563982    0.20132983 -0.5810045  ...  0.31268626  0.8904828
   0.00378721]
 [ 0.5610509   0.20047116 -0.28053388 ...  0.33581167  0.8375484
  -0.03704382]
 ...
 [ 0.49563783  0.26006433 -0.14971872 ...  0.313502    0.7296397
  -0.01058379]
 [ 0.49839425  0.24982984 -0.11155556 ...  0.31379032  0.78602326
  -0.00598459]
 [ 0.5072653   0.24654277 -0.06613079 ...  0.30803528  0.8276688
   0.00090764]]


In [14]:
update_sql = """
    UPDATE sign_videos SET verified = 1, true_label = 'hello' WHERE id = 1
"""
cursor.execute(update_sql)
conn.commit()

In [30]:
v_name, blob = res[0]

In [31]:
with open(v_name, "wb") as video_file:
    video_file.write(blob)

In [32]:
import numpy as np


a = np.array([1, 2, 3])
np.save('my_array.npy', a)

In [33]:
with open('my_array.npy', "rb") as numpy_blob:
    array_num = numpy_blob.read()

In [35]:
with open('test.npy', 'wb') as nb:
    nb.write(array_num)

In [36]:
loaded_data = np.load('test.npy')

In [37]:
loaded_data

array([1, 2, 3])

In [16]:
conn.close()